## Парсинг

In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import re

C:\Users\annat\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\annat\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Для начала спарсим все сайты отелей, которые мы собираемся использовать для анализа. Все сайты сохранены в переменной `hotels`.

In [2]:
u = 'https://ostrovok.ru/hotel/russia/sochi/'
all_url = []

page = 0
next_page = '?page='

for i in range(0, 401):
    if page != 0:
        url = u + next_page + str(page)
    else:
        url = u
    page += 1
    all_url.append(url)

In [3]:
# парсинг сайтов каждого отеля
dom = 'https://ostrovok.ru'
hotels = [] # лист, куда сохраняются сайты отелей

for i in all_url:
    response = requests.get(url=i)
    soup = BeautifulSoup(response.content, 'html.parser')
    div = soup.find_all("div", {"class": "HotelCard_wrapTitle__t742O"})

    for k in range(0, len(div)-1):
        hotels.append(dom + str(div[k]).split('href="')[-1].split('" target=')[0])

hotels = np.unique(np.array(hotels))

In [16]:
hotels = np.unique(np.array(hotels))

Всего получилось спарсить 7543 отеля.

In [5]:
hotels.shape

(7543,)

Теперь проитерируемся по всем сайтам, чтобы получить необходимую информацию по каждому отелю.

In [2]:
# тут если отели импортируем из файлика а не парсим 

f = open('C:\\Users\\annat\\hotels.txt', 'r')
hotels = f.read()

In [3]:
hotels = np.array(hotels[1:-1].replace("'", '').split(', '))

In [5]:
# листы, куда сохраняются все характ-ки
perks = []
amenities = []
dist_sea = []
dist_city = []
stars = []
feedb = []
price = []
feedb_n = []
num_rooms = []
hotels_good = []
pool = []
kids = []
anim = []

In [6]:
# код для парсинга

for i in range(len(hotels)-1):
    s = hotels[i]
    response_h = requests.get(url=s)
    soup_h = BeautifulSoup(response_h.content, 'html.parser')
    div_h = soup_h.find_all("div", {"class": "Perks_amenities__RC9_b"})
    if len(div_h) < 1:
        continue
    perks_1 = []
    perks_tag = div_h[0].find_all('li')
    for k in perks_tag:
        perks_1.append(k.string)
    perks.append(perks_1)
    
    amenities_1 = []
    div_am = soup_h.find_all("div", {"class": "Amenities_list__pYDjd"})
    amenities_tag = div_am[0].find_all("div", {"class": "Amenities_amenityName__a_l1_"})
    for k in amenities_tag:
        amenities_1.append(k.string)
    amenities.append(amenities_1)
    
    dist_tag = soup_h.find("div", {"class": "HotelHeader_distances__FS_8W"}).find_all("span")
    if str(dist_tag[1]).find('центр') != -1:
        dist_city.append(dist_tag[0].get_text())
    elif len(dist_tag) >= 4:
        if (str(dist_tag[3]).find('центр') != -1):
            dist_city.append(dist_tag[2].get_text())
    else:
        dist_city.append(np.nan)
    
    crit = 0
    list_beach = soup_h.find("div", {"class": "Pois_pois__9b9fn"}).find_all("span", {"class": ""})[::2]
    list_beach_dist = soup_h.find("div", {"class": "Pois_pois__9b9fn"}).find_all("span", {"class": "Pois_distance__LdxIf"})
    for k in range(len(list_beach)):
        if list_beach[k].find('Пляж') != -1:
            dist_sea.append(list_beach_dist[k].get_text())
            crit = 1
            break
    if crit == 0:
        dist_sea.append(np.nan)

    inf_tag = soup_h.find_all('script')[1]
    inf_string = inf_tag.get_text().split('\n')
    ind_star = inf_string.index('  "starRating": {')
    stars.append(re.findall(r'\d+', inf_string[ind_star+2])[0])

    ind_feed = inf_string.index('  "aggregateRating": {')
    if re.findall(r'\.', inf_string[ind_feed+3]) == []:
        feedb.append(re.findall(r'\d+', inf_string[ind_feed+3])[0])
    else:
        feedb.append(re.findall(r'\d+\.\d+', inf_string[ind_feed+3])[0])

    ind_price = inf_string.index('  "makesOffer": {')
    price.append(re.findall(r'\d+', inf_string[ind_price+2])[0])
    
    feed_n_str = soup_h.find("p", {"class": "Rating_reviews__3jVPl"})
    if feed_n_str == None:
        feedb_n.append(0)
    else:
        feedb_n.append(re.findall(r'\d+', feed_n_str.get_text())[0])
    
    num_room = soup_h.find_all("div", {"class":"Facts_content__ua_a4"})
    check_rooms = 0
    for k in range(len(num_room)):
        if num_room[k].get_text().find('номеров') != -1:
            num_rooms.append(re.findall(r'\d+', num_room[k].get_text())[0])
            check_rooms = 1
    if check_rooms != 1:
        num_rooms.append(np.nan)
    
    hotels_good.append(hotels[i])

    pool_n = 0
    kids_n = 0
    anim_n = 0
    amen_list = soup_h.find_all("p", {"class":"Amenities_groupTitle__aDVIi"})
    for k in range(len(amen_list)-1):
        if amen_list[k].get_text().find('ассейн') != -1:
            pool_n = 1
        elif amen_list[k].get_text().find('Дети') != -1:
            kids_n = len(soup_h.find("div", {"class":"Amenities_group__X5Qd7 Amenities_group_has_kids__eSVkf"}).find_all("div", {"class":"Amenities_amenityName__a_l1_"}))
        elif amen_list[k].get_text().find('Животные') != -1:
            if soup_h.find("div", {"class":"Amenities_group__X5Qd7 Amenities_group_has_pets__VU3lp"}).find("div", {"class":"Amenities_amenityName__a_l1_"}).get_text().find('не') == -1:
                anim_n = 1
    pool.append(pool_n)
    kids.append(kids_n)
    anim.append(anim_n)

In [10]:
# все характ-ки преобразуем в датафрейм
columns = ['web', 'perks', 'amenities', 'dist_sea', 'dist_city', 'stars', 'feedb', 'price', 'feedb_n', 'num_rooms', 'pool', 'kids', 'anim']
all_char = pd.DataFrame([hotels_good, perks, amenities, dist_sea, dist_city, stars, feedb, price, feedb_n, num_rooms, pool, kids, anim]).T
all_char.columns = columns
all_char.head()

,web,perks,amenities,dist_sea,dist_city,stars,feedb,price,feedb_n,num_rooms,pool,kids,anim,location
0,https://ostrovok.ru/hotel/russia/adler/mid1000...,"['Бесплатный интернет', 'Трансфер', 'Бар или р...","['Кондиционер', 'Круглосуточная стойка регистр...",140 м,1.3 км,3,9.1,6326,89,37.0,0,0,0,adler
1,https://ostrovok.ru/hotel/russia/adler/mid1000...,"['Бесплатный интернет', 'Подходит для детей', ...","['Круглосуточная стойка регистрации', 'Стираль...",280 м,825.6 м,0,5.4,4710,2,NaN,0,0,0,adler
2,https://ostrovok.ru/hotel/russia/adler/mid1000...,"['Бесплатный интернет', 'Подходит для детей', ...","['Кондиционер', 'Круглосуточная стойка регистр...",590 м,856 м,0,8.2,1863,34,12.0,0,2,0,adler
3,https://ostrovok.ru/hotel/russia/adler/mid1001...,"['Бесплатный интернет', 'Трансфер', 'Парковка'...","['Места для курения', 'Телевизор в лобби', 'Ме...",180 м,6.1 км,0,6.7,2250,25,NaN,1,0,0,adler
4,https://ostrovok.ru/hotel/russia/adler/mid1021...,"['Бесплатный интернет', 'Парковка', 'Подходит ...","['Кондиционер', 'Круглосуточная стойка регистр...",350 м,1 км,0,8.0,3618,49,20.0,0,1,0,adler


In [11]:
all_char.shape

(7448, 14)

In [13]:
# сохраняем итоговую таблицу в файл csv
all_char.to_csv('hotels_all_data', sep='\t', encoding='utf-8')

## Обработка данных

In [17]:
all_data = pd.read_csv('https://raw.githubusercontent.com/pavleke/hotels/main/hotels_all_data', on_bad_lines='skip', sep='\t').drop('Unnamed: 0', axis=1)
all_data.shape

(7448, 14)

#### Перевод из строки в список и добавление дамми-переменных

Так как в csv-файле списки со всеми удобствами сохранились как строки, то напишем функцию, которая переведет эти строки обратно в списки.

In [15]:
# функция для перевода строки в лист

def str_to_list(str_):
    list_ = np.array(str_[1:-1].replace("'", "").split(', '))
    return(list_)

1) `perks`

In [18]:
new_perks_arr = np.array([])
new_perks = []
for i in all_data['perks']:
    new_perks_arr = np.append(new_perks_arr, str_to_list(i))
    new_perks.append(str_to_list(i))

Теперь нужно перевести все категориальные переменные в дамми-переменные. Начнем с `perks`.

In [19]:
uniq_perks = np.unique(new_perks_arr)
df_perks = pd.DataFrame(columns=uniq_perks)

for k in range(uniq_perks.shape[0]):
    n_perks = np.array([])
    for i in new_perks:
        if np.isin(i, uniq_perks[k]).sum() != 0:
            n_perks = np.append(n_perks, 1)
        else:
            n_perks = np.append(n_perks, 0)
    df_perks[uniq_perks[k]] = n_perks

In [20]:
df_perks = df_perks.drop('None', axis=1)

In [21]:
df_perks['Бар или ресторан'] = df_perks['Бар или ресторан'] + df_perks['Bar or restaurant']
df_perks['Бассейн'] = df_perks['Бассейн'] + df_perks['Swimming Pool']
df_perks['Бесплатный интернет'] = df_perks['Бесплатный интернет'] + df_perks['Free Internet']
df_perks['Кондиционер'] = df_perks['Кондиционер'] + df_perks['Air-conditioning']
df_perks['Парковка'] = df_perks['Парковка'] + df_perks['Parking']
df_perks['Пляж рядом'] = df_perks['Пляж рядом'] + df_perks['Beach nearby']
df_perks['Подходит для детей'] = df_perks['Подходит для детей'] + df_perks['Suitable for children']
df_perks['Спа-услуги'] = df_perks['Спа-услуги'] + df_perks['Spa Services']
df_perks['Трансфер'] = df_perks['Трансфер'] + df_perks['Transfer']
df_perks = df_perks.drop(['Bar or restaurant', 'Swimming Pool', 'Free Internet', 'Air-conditioning', 'Parking', 'Beach nearby', 'Suitable for children', 'Spa Services', 'Transfer'], axis=1)

In [23]:
df_perks.head()

,Бар или ресторан,Бассейн,Бесплатный интернет,Джакузи,Для гостей с ограниченными возможностями,Кондиционер,Конференц-зал,Кухня,Можно курить,Парковка,Пляж рядом,Подходит для детей,Разрешено с домашними животными,Спа-услуги,Трансфер,Фитнес
0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [209]:
# сохраним бинарные perks в отдельный файл csv
df_perks.to_csv('perks_bin', sep='\t')

2) `amenities`

Теперь превратим `amenities` в списки, а затем в дамми-переменные.

In [25]:
new_amen_arr = np.array([])
new_amen = []
for i in all_data['amenities']:
    new_amen_arr = np.append(new_amen_arr, str_to_list(i))
    new_amen.append(str_to_list(i))

In [26]:
uniq_amen = np.unique(new_amen_arr)
df_amen = pd.DataFrame(columns=uniq_amen)

for k in range(uniq_amen.shape[0]):
    n_amen = np.array([])
    for i in new_amen:
        if np.isin(i, uniq_amen[k]).sum() != 0:
            n_amen = np.append(n_amen, 1)
        else:
            n_amen = np.append(n_amen, 0)
    df_amen[uniq_amen[k]] = n_amen

In [30]:
uniq_amen.shape

(354,)

Так как уникальных `amenities` очень много (354 характ-ки), то создадим дамми-переменные только для наиболее часто повторяющихся значений.

In [47]:
pd.DataFrame([uniq_amen, np.unique(new_amen_arr, return_counts=True)[1]], index=['amen', 'counts']).T.sort_values(by='counts', ascending=False)[:10]

,amen,counts
177,Кондиционер,4725
108,Бесплатный Wi-Fi,4383
290,Телевизор,4293
336,Холодильник,3887
131,Гладильные принадлежности,3704
219,Отопление,3679
351,на русском,3452
326,Утюг,3380
284,Стиральная машина,3372
206,Номера для некурящих,2945


In [48]:
popular_amen = ['Телевизор', 
                'Breakfast', 
                'Cable TV', 
                'Fridge', 
                'Heating', 
                'Iron', 
                'Iron and board', 
                'Ironing', 
                'Non-smoking rooms', 
                'Pets allowed', 
                'Smoke-free property', 
                'TV', 
                'Television in lobby', 
                'Гладильные принадлежности',
                'Гладильные принадлежности (по запросу)', 
                'Гладильные услуги', 
                'Завтрак', 'Завтрак «шведский стол»', 
                'Завтрак в номер', 
                'Кабельное телевидение', 
                'Курение запрещено на всей территории', 
                'Курение разрешено', 
                'Номера для некурящих', 
                'Отель для некурящих', 
                'Отопление', 
                'Телевизор в лобби', 
                'Телевизор с плоским экраном',
                'Удобства для гостей с ограниченными физическими возможностями',
                'Упакованные завтраки/обеды', 
                'Утюг',
                'Холодильник']

df_amen_bin = pd.DataFrame()

df_amen_bin['tv_bin'] = df_amen['Телевизор'] + df_amen['Cable TV'] + df_amen['TV'] + df_amen['Television in lobby'] + df_amen['Кабельное телевидение'] + df_amen['Телевизор в лобби'] + df_amen['Телевизор с плоским экраном']
df_amen_bin['brekf_bin'] = df_amen['Breakfast'] + df_amen['Завтрак'] + df_amen['Завтрак «шведский стол»'] + df_amen['Завтрак в номер'] + df_amen['Упакованные завтраки/обеды']
df_amen_bin['fridge_bin'] = df_amen['Fridge'] + df_amen['Холодильник']
df_amen_bin['heat_bin'] = df_amen['Heating'] + df_amen['Отопление']
df_amen_bin['iron_bin'] = df_amen['Iron'] + df_amen['Iron and board'] + df_amen['Ironing'] + df_amen['Гладильные принадлежности'] + df_amen['Гладильные принадлежности (по запросу)'] + df_amen['Гладильные услуги'] + df_amen['Утюг']
df_amen_bin['smokefree_bin'] = df_amen['Smoke-free property'] + df_amen['Non-smoking rooms'] + df_amen['Курение запрещено на всей территории'] + abs(df_amen['Курение разрешено'] - 1) + df_amen['Номера для некурящих'] + df_amen['Отель для некурящих']
df_amen_bin['invalid_bin'] = df_amen['Удобства для гостей с ограниченными физическими возможностями']

In [49]:
df_amen_bin = np.sign(df_amen_bin)

In [50]:
df_amen_bin.head()

,tv_bin,brekf_bin,fridge_bin,heat_bin,iron_bin,smokefree_bin,invalid_bin
0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,1.0,0.0,1.0,1.0,1.0,1.0,0.0
3,1.0,0.0,1.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,1.0,1.0,1.0,0.0


In [ ]:
# сохраним бинарные amenities в отдельный файл csv
df_amen_bin.to_csv('amen_bin', sep='\t')

#### Добавление местоположения отеля

Отметим, что на стоимость проживания также может влиять расположение отеля в отдельном районе г. Сочи или в населенном пункте городской агломерации. Поэтому в качестве признака добавим метоположение отеля.

In [51]:
all_data['web'][0].split('/')[5]
location = np.array([])
for i in all_data['web']:
    location = np.append(location, i.split('/')[5])
    
all_data['location'] = location

In [304]:
# сохраним все данные в отдельный файл csv
all_data.to_csv('hotels_all_data', sep='\t', encoding='utf-8')

#### Обработка расстояний

Следующий шаг - перевод расстояний из километров в метры и из строк в числа.

In [52]:
df = pd.read_csv('https://raw.githubusercontent.com/pavleke/hotels/main/hotels_all_data', on_bad_lines='skip', sep='\t').drop('Unnamed: 0', axis=1)
df.head()

,web,perks,amenities,dist_sea,dist_city,stars,feedb,price,feedb_n,num_rooms,pool,kids,anim,location
0,https://ostrovok.ru/hotel/russia/adler/mid1000...,"['Бесплатный интернет', 'Трансфер', 'Бар или р...","['Кондиционер', 'Круглосуточная стойка регистр...",140 м,1.3 км,3,9.1,6326,89,37.0,0,0,0,adler
1,https://ostrovok.ru/hotel/russia/adler/mid1000...,"['Бесплатный интернет', 'Подходит для детей', ...","['Круглосуточная стойка регистрации', 'Стираль...",280 м,825.6 м,0,5.4,4710,2,NaN,0,0,0,adler
2,https://ostrovok.ru/hotel/russia/adler/mid1000...,"['Бесплатный интернет', 'Подходит для детей', ...","['Кондиционер', 'Круглосуточная стойка регистр...",590 м,856 м,0,8.2,1863,34,12.0,0,2,0,adler
3,https://ostrovok.ru/hotel/russia/adler/mid1001...,"['Бесплатный интернет', 'Трансфер', 'Парковка'...","['Места для курения', 'Телевизор в лобби', 'Ме...",180 м,6.1 км,0,6.7,2250,25,NaN,1,0,0,adler
4,https://ostrovok.ru/hotel/russia/adler/mid1021...,"['Бесплатный интернет', 'Парковка', 'Подходит ...","['Кондиционер', 'Круглосуточная стойка регистр...",350 м,1 км,0,8.0,3618,49,20.0,0,1,0,adler


In [53]:
df.loc[df['dist_sea'].str.contains('км')==True, 'dist_sea_km'] = 1
df.loc[df['dist_sea'].str.contains('км')==False, 'dist_sea_km'] = 0
df.loc[df['dist_city'].str.contains('км')==True, 'dist_city_km'] = 1
df.loc[df['dist_city'].str.contains('км')==False, 'dist_city_km'] = 0

df[['dist_sea', 'd_s']] = df['dist_sea'].str.split(' ', n=1, expand=True)
df[['dist_city', 'd_c']] = df['dist_city'].str.split(' ', n=1, expand=True)
df.drop(['d_s', 'd_c'], axis=1, inplace=True)
df['dist_sea'] = df['dist_sea'].astype(float)
df['dist_city'] = df['dist_city'].astype(float)
df.loc[df['dist_sea_km']==1, 'dist_sea'] = df['dist_sea'] * 1000
df.loc[df['dist_city_km']==1, 'dist_city'] = df['dist_city'] * 1000

#### Добавление количества удобств

Добавим новые признаки `perks_n` и `amenities_n` - они показывают количество основных удобств и дополнительных удобств, соответственно.

In [54]:
df['perks'] = df['perks'].str.replace(', None', '')
df['amenities'] = df['amenities'].str.replace(', None', '')

In [55]:
df['perks_n'] = df['perks'].str.split(',').apply(lambda x: len(x))
df['amenities_n'] = df['amenities'].str.split(',').apply(lambda x: len(x))

In [57]:
df.head()

,web,perks,amenities,dist_sea,dist_city,stars,feedb,price,feedb_n,num_rooms,pool,kids,anim,location,dist_sea_km,dist_city_km,perks_n,amenities_n
0,https://ostrovok.ru/hotel/russia/adler/mid1000...,"['Бесплатный интернет', 'Трансфер', 'Бар или р...","['Кондиционер', 'Круглосуточная стойка регистр...",140.0,1300.0,3,9.1,6326,89,37.0,0,0,0,adler,0.0,1.0,5,24
1,https://ostrovok.ru/hotel/russia/adler/mid1000...,"['Бесплатный интернет', 'Подходит для детей', ...","['Круглосуточная стойка регистрации', 'Стираль...",280.0,825.6,0,5.4,4710,2,NaN,0,0,0,adler,0.0,0.0,4,13
2,https://ostrovok.ru/hotel/russia/adler/mid1000...,"['Бесплатный интернет', 'Подходит для детей', ...","['Кондиционер', 'Круглосуточная стойка регистр...",590.0,856.0,0,8.2,1863,34,12.0,0,2,0,adler,0.0,0.0,5,38
3,https://ostrovok.ru/hotel/russia/adler/mid1001...,"['Бесплатный интернет', 'Трансфер', 'Парковка'...","['Места для курения', 'Телевизор в лобби', 'Ме...",180.0,6100.0,0,6.7,2250,25,NaN,1,0,0,adler,0.0,1.0,5,17
4,https://ostrovok.ru/hotel/russia/adler/mid1021...,"['Бесплатный интернет', 'Парковка', 'Подходит ...","['Кондиционер', 'Круглосуточная стойка регистр...",350.0,1000.0,0,8.0,3618,49,20.0,0,1,0,adler,0.0,1.0,5,28


In [58]:
df.shape

(7448, 18)

In [61]:
# сохраним все данные в отдельный файл csv
df.to_csv('hotels_without_bin.csv', sep='\t')#, encoding='utf-8')

### Дополнительно

Перебор коэффициентов при логарифме суммы (VIF):

In [ ]:
vif_w_summ = [] 
r2_w_summ = [] 
weights = [] 
res_w = pd.DataFrame(columns=['weights', 'vif', 'r2']) 
#X_log_am_sum_dist['sum_dist_w'] = 
for i in np.arange(0.1, 1, 0.1): 
    w = {'sea': round(i, 1), 'city': round((1-i), 1)} 
    X_weighted = X_log_am 
    X_weighted['sum_dist_w'] = np.log(X_1['dist_city'] * w['city'] + X_1['dist_sea'] * w['sea']) 
    X_weighted = X_weighted.drop(['dist_sea', 'dist_city', 'Пляж рядом'], axis=1) 
    vif_w = variance_inflation_factor(X_weighted.values, 25) 
    X_weighted_с = sm.add_constant(X_weighted, prepend=False) 
    r2_w = sm.OLS(y_log, X_weighted_с).fit().rsquared 
    vif_w_summ.append(vif_w) 
    r2_w_summ.append(r2_w) 
    weights.append(w) 
 
res_w['weights'] = weights 
res_w['vif'] = vif_w_summ 
res_w['r2'] = r2_w_summ 
 
res_w

Создание новой переменной `amen_only` (мультиколлинеарность):

In [ ]:
df_sochi = df_all_w.loc[df_all_w.location == 'sochi'].reset_index() 
all_new_amen = [] 
for i in range(df_sochi.shape[0]): 
    amen_ = df_sochi.amenities.loc[i][1:-1].replace("'", '').split(', ') 
    perks_ = df_sochi.perks.loc[i][1:-1].replace("'", '').split(', ') 
    new_amen = len(amen_) 
    string = df_sochi.amenities.loc[i].lower().replace('кафе', 'бар') 
    for k in perks_: 
        k = k.lower() 
        if 'интернет' in k: 
            if 'интернет' in string: 
                new_amen -= 1 
            k = 'wi-fi' 
        if 'детей' in k: 
            k = 'дет' 
        if 'бар' in k: 
            k = 'бар' 
        if 'животными' in k: 
            k = 'живот' 
        if 'курить' in k: 
            k = 'кур' 
        if 'спа' in k: 
            k = 'спа' 
        if 'огранич' in k: 
            k = 'огранич' 
        if 'пляж рядом' in k: 
            new_amen -= 1 
        if k in string: 
            new_amen -= string.count(k) 
        if ('ресторан' in string) and (k == 'бар'): 
            new_amen -= string.count('ресторан') 
        if ('тренаж' in string) and ('фит' in k): 
            new_amen -= 1 
        if k == 'спа': 
            if 'красот' in string: 
                new_amen -= string.count('красот') 
            if 'сауна' in string: 
                new_amen -= string.count('сауна') 
            if 'массаж' in string: 
                new_amen -= string.count('массаж') 
            if 'баня' in string: 
                new_amen -= string.count('баня') 
        if 'отсутствие лифтов' in string: 
            new_amen -= 1 
    all_new_amen.append(new_amen) 
 
X_log_am = X_log.copy() 
X_log_am['amen_only'] = all_new_amen 
X_log_am = X_log_am.drop(['amenities_n', 'perks_n'], axis=1)

Преобразование двух переменных в одну:

In [ ]:
df_for_model.loc[(df_for_model['kids'] >= 1.0) | (df_for_model['Подходит для детей'] == 1.0), 'Подходит для детей'] = 1 
df_for_model.loc[(df_for_model['kids'] == 0.0) & (df_for_model['Подходит для детей'] == 0.0), 'Подходит для детей'] = 0 
df_for_model.loc[(df_for_model['pool'] == 1.0) | (df_for_model['Бассейн'] == 1.0), 'Бассейн'] = 1 
df_for_model.loc[(df_for_model['pool'] == 0.0) & (df_for_model['Бассейн'] == 0.0), 'Бассейн'] = 0 
df_for_model.loc[(df_for_model['anim'] == 1.0) | (df_for_model['Разрешено с домашними животными'] == 1.0), 'Разрешено с домашними животными'] = 1 
df_for_model.loc[(df_for_model['anim'] == 0.0) & (df_for_model['Разрешено с домашними животными'] == 0.0), 'Разрешено с домашними животными'] = 0 
df_for_model.loc[(df_for_model['invalid_bin'] == 1.0) | (df_for_model['Для гостей с ограниченными возможностями'] == 1.0), 'Для гостей с ограниченными возможностями'] = 1 
df_for_model.loc[(df_for_model['invalid_bin'] == 0.0) & (df_for_model['Для гостей с ограниченными возможностями'] == 0.0), 'Для гостей с ограниченными возможностями'] = 0 
df_for_model.loc[(df_for_model['smokefree_bin'] == 0.0) | (df_for_model['Можно курить'] == 1.0), 'Можно курить'] = 1 
df_for_model.loc[(df_for_model['smokefree_bin'] == 1.0) & (df_for_model['Можно курить'] == 0.0), 'Можно курить'] = 0
df_for_model = df_for_model.drop(['kids', 'dist_sea_km', 'dist_city_km', 'pool', 'anim', 'invalid_bin', 'smokefree_bin'], axis=1)